# ETF Data Exploration

In [1]:
from etf_data_analysis_dashboard.config import RAW_DATA_PATH, NUM_YEARS
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [23]:
px.defaults.template = "plotly_white"

In [16]:
spy = pd.read_csv(
    RAW_DATA_PATH / "data.csv",
    names=["Date", "Close", "High", "Low", "Open", "Volume"],
    parse_dates=["Date"],
    index_col="Date",
    skiprows=3,
)

In [4]:
spy.head()

,Close,High,Low,Open,Volume
Date,,,,,
2023-03-21,389.442017,389.930149,386.191036,387.811640,91524200
2023-03-22,382.803406,392.937045,382.764377,389.266308,111746600
2023-03-23,383.838287,389.813027,381.085212,385.712700,119351300
2023-03-24,386.357025,386.444885,380.157734,382.539824,107682400
2023-03-27,387.079468,389.451815,386.171548,388.670785,74010400


In [5]:
spy.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 499 entries, 2023-03-21 to 2025-03-17
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Close   499 non-null    float64
 1   High    499 non-null    float64
 2   Low     499 non-null    float64
 3   Open    499 non-null    float64
 4   Volume  499 non-null    int64  
dtypes: float64(4), int64(1)
memory usage: 23.4 KB


## Plot time series

In general, we are interested at the closing price of each day (column `close`). As a first step, I will plot the closing price evolution (in USD).

In [44]:
title = f"SPY Closing Price Evolution in the last {NUM_YEARS} years"
px.line(spy, x=spy.index, y="Close", title=title).update_layout(
    xaxis_title="", yaxis_title="Close Price (in $)"
)

## Data preprocessing requirements

- Fix date index: How do we deal with weekends, holidays, etc. ?
    1. include all dates between `START_DATE` and `END_DATE` of the time series
    2. forward fill `Close` price
    3. include `is_weekend` and `is_holiday` bool flags
- Extract features using a library like `tsfresh`

In [57]:
spy_monthly = spy.resample('W').last()

In [59]:
title = f"SPY Closing Price Evolution by Month in the last {NUM_YEARS} years"
px.line(spy, x=spy.index, y="Close", title=title).update_layout(
    xaxis_title="", yaxis_title="Close Price (in $)"
)

In [7]:
def clean_data(df: pd.DataFrame) -> pd.DataFrame:
    """
    Perform the data cleaning steps on the raw DataFrame.

    The goal is to eliminate any missing values and to ensure that the data is ready for analysis and further processing.

    Parameters
    ----------
    df : pd.DataFrame
        the raw data extracted from the source (yfinance, Alpha Vantage, etc.)

    Returns
    -------
    pd.DataFrame
        complete DataFrame
    """
    temp = df.copy()
    temp.columns = temp.columns.str.lower()
    temp.index.name = temp.index.name.lower()
    all_dates = pd.date_range(start=temp.index.min(), end=temp.index.max(), freq="D")
    temp = temp.reindex(all_dates).rename_axis("date")
    temp["close"] = temp["close"].ffill()

    return temp[["close"]]

In [8]:
spy_cleaned = clean_data(spy)

In [9]:
spy_cleaned.head()

,close
date,
2023-03-21,389.442017
2023-03-22,382.803406
2023-03-23,383.838287
2023-03-24,386.357025
2023-03-25,386.357025


In [25]:
title = f"SPY Closing Price Evolution in the last {NUM_YEARS} years"
subtitle = "Including weekends and holidays (filled with the last available value)"
px.line(
    spy_cleaned, x=spy_cleaned.index, y="close", title=title, subtitle=subtitle
).update_layout(xaxis_title="", yaxis_title="Close Price (in $)")

## Data preprocessing requirements

- Fix date index: How do we deal with weekends, holidays, etc. ?
    1. include all dates between `START_DATE` and `END_DATE` of the time series
    2. forward fill `Close` price
    3. include `is_weekend` and `is_holiday` bool flags
- Extract features using a library like `tsfresh`

In [33]:
def make_features(df:pd.DataFrame)->pd.DataFrame:
    """
    Create new features from the existing ones

    Parameters
    ----------
    df : pd.DataFrame
        _description_

    Returns
    -------
    pd.DataFrame
        enhanced DataFrame
    """
    temp = df.copy()

    # Daily returns
    temp["daily_return"] = temp["close"].pct_change().fillna(0)
    
    # Extract date features
    temp["day_of_week"] = temp.index.dayofweek
    temp["day_of_month"] = temp.index.day
    temp["month"] = temp.index.month
    temp["year"] = temp.index.year

    #
    return temp

In [34]:
spy_enhanced = make_features(spy_cleaned)

In [35]:
title = f"SPY Daily Returns Evolution in the last {NUM_YEARS} years"
subtitle = "Including weekends and holidays"
px.line(
    spy_enhanced, x=spy_enhanced.index, y="daily_return", title=title, subtitle=subtitle
).update_layout(yaxis_title="Daily Return (%)")

In [38]:
px.histogram(spy_enhanced, x="daily_return", nbins=100, title="Distribution of Daily Returns")

In [82]:
def preprocess_data():
    ...